In [4]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from plotting_functions import *
from analysis_functions import *
import cmocean
import matplotlib.colors as mcolors
import matplotlib.dates as mdates

plt.style.use('../misc/acgc.mplstyle')

color_dict = {'brown':  (202, 197, 195),
              'red':    (171, 103, 87), 
              'orange': (204, 128, 83), 
              'yellow': (240, 179, 96),
              'green':  (108, 138, 105),
              'blue':   (137, 156, 175)}
for key in color_dict:
    r, g, b = color_dict[key]
    color_dict[key] = (r / 255., g / 255., b / 255.)

domain_names   = ['Alaska','YKD','IKU']
magnitudes     = ['low', 'mid', 'high']
f_TGMs         = ['low', 'mid', 'high']
plume_heights  = ['surface', 'low', 'mid', 'high']
emission_types = ['Baseline','Fire (TGM)', 'Fire (TPM)']

domains = get_domain_bounds()

In [5]:
# ----------------------------------------------
# Flag for whether to mask out the ocean
# ----------------------------------------------

# - flag for whether to mask out the ocean
bool_mask_ocean = True #True
# - flag for whether to exclude ocean Hg0 uptake from 'Total_Hg_Deposition'
bool_exclude_ocean_Hg0_uptake = False

if bool_mask_ocean:
    area_var = 'area [m2]'
    file_suffix = '_land_only'
else:
    area_var = 'area [m2]'
    file_suffix = ''
# ----------------------------------------------

In [ ]:
# -- display summary of land/ocean/lake landcover classification breakdown for each domain
tmp = xr.open_mfdataset('../model_output/v14.3/Baseline_deposition.nc')
masks = xr.open_mfdataset('../misc/masks.nc')
# merge tmp['AREA'] with masks
masks = xr.merge([masks, tmp['AREA']])
for key, d in domains.items():
    print(key)
    sel = masks.sel( lat=slice(d['lat_min'], d['lat_max']), lon=slice(d['lon_min'], d['lon_max']) )
    area_ocean = (sel['Met_FROCEAN'] * sel['AREA']).sum().values.item()
    area_land  = (sel['Met_FRLAND'] * sel['AREA']).sum().values.item()
    area_lake  = (sel['Met_FRLAKE'] * sel['AREA']).sum().values.item()
    area_total = sel['AREA'].sum().values.item()
    print('total area [m2]', area_total)
    print('land area [m2]', area_total - area_ocean)
    print('f_ocean:', np.round(area_ocean/area_total, 2))
    print('f_lake: ', np.round((area_lake/area_total), 2))
    print('f_land: ', np.round((area_land/area_total), 2))
    print('f_other:', np.round(1-(area_ocean+area_lake+area_land)/area_total, 2))

In [7]:
# Three sets of tables [Alaska domain, YKD domain, IKU domain]
# each table has deposition for each month of the year in [kg/mo, ug/m2/mo]
domains = get_domain_bounds()

def make_table(ds, domain_name:str, emission_scenario_name:str, dep_var='Total_Hg_Deposition', area_var='area [m2]'):

    # subset ds to domain bounds (lat, lon)
    ds = ds.sel(lat=slice(domains[domain_name]['lat_min'], domains[domain_name]['lat_max']), lon=slice(domains[domain_name]['lon_min'], domains[domain_name]['lon_max']))

    total_dep = get_total_dep(ds, dep_var=dep_var)
    table = pd.DataFrame(total_dep)
    table['total deposition [ug/m2/mo]'] = (table[f'{dep_var} [kg/mo]']/table['area [m2]'])*1e9
    table['emission scenario'] = emission_scenario_name
    table['domain name'] = domain_name
    for key in domains[domain_name]:
        #print(key)
        table[key] = domains[domain_name][key]
    table['year']  = table['time'].dt.year
    table['month'] = table['time'].dt.month
    # rename and reorder columns
    table = table.rename(columns={'time':'date', f'{dep_var} [kg/mo]':'total deposition [kg/mo]', 'lat_min':'latitude min', 'lat_max':'latitude max', 'lon_min':'longitude min', 'lon_max':'longitude max'})
    table = table[['emission scenario', 'domain name', 'year', 'month', 'date', 'latitude min', 'latitude max', 'longitude min', 'longitude max', area_var, 'total deposition [kg/mo]', 'total deposition [ug/m2/mo]']]
    return table

In [8]:
def scale_based_on_emission_uncertainty(table, domain_name:str, emission_scenario:str, magnitude_tag:str, f_TGM_tag:float):
    """ Scale deposition according to the uncertainty in the emission magnitude and fraction TGM. 
        This needs to be run separately for TGM and TPM output. 
    
    Parameters
    ----------
    table : pd.DataFrame
        deposition table
    domain_name : str
        domain name in ['Alaska', 'YKD', 'IKU']
    emission_scenario : str
        emission scenario in ['Baseline', 'Fire (TGM)', 'Fire (TPM)']
    magnitude_tag : str
        magnitude tag in ['low', 'mid', 'high']
    f_TGM_tag : float
        fraction TGM tag in ['low', 'mid', 'high']
    
    Returns
    -------
    pd.DataFrame
        scaled deposition table
    """

    f_TGM_range = {'mid': 0.935, 'low': 0.935, 'high': 0.992}
    f_TGM = f_TGM_range[f_TGM_tag]

    magnitude_range = {'mid': 556, 'low':164, 'high':1138}
    magnitude = magnitude_range[magnitude_tag]

    # call function to subset table to only include rows that match the domain name and emission scenario
    out = subset_table(table, match_dict={'domain name':domain_name, 'emission scenario':emission_scenario})

    if 'TGM' in emission_scenario:
        base_E = 565.9 # emission [kg] used in the model simulations
        scale_factor = scale_ds(f_TGM, magnitude)['scale_TGM']
        out['emission [kg]'] = base_E*scale_factor
    elif 'TPM' in emission_scenario:
        base_E = 21.4 # emission [kg] used in the model simulations
        scale_factor = scale_ds(f_TGM, magnitude)['scale_TPM']
        out['emission [kg]'] = base_E*scale_factor
    else:
        raise ValueError('emission scenario must be Fire (TGM) or Fire (TPM)')
    
    out['total deposition [kg/mo]']    = out['total deposition [kg/mo]']*scale_factor
    out['total deposition [ug/m2/mo]'] = out['total deposition [ug/m2/mo]']*scale_factor

    out['magnitude label'] = magnitude_tag
    out['fraction TGM label'] = f_TGM_tag

    return out

In [9]:
# --------------------------------------------------------------------------------------------------------------
# load GEOS-Chem output
# --------------------------------------------------------------------------------------------------------------
baseline = xr.open_mfdataset('../model_output/v14.3/Baseline_deposition.nc')
IKU_fire_TGM = xr.open_mfdataset('../model_output/v14.3/Fire_TGM_deposition.nc')
IKU_fire_TPM = xr.open_mfdataset('../model_output/v14.3/Fire_TPM_deposition.nc')
IKU_fire_TGM_high_FT = xr.open_mfdataset('../model_output/v14.3/Fire_TGM_high_FT_deposition.nc')
IKU_fire_TPM_high_FT = xr.open_mfdataset('../model_output/v14.3/Fire_TPM_high_FT_deposition.nc')
IKU_fire_TGM_low_FT  = xr.open_mfdataset('../model_output/v14.3/Fire_TGM_low_FT_deposition.nc')
IKU_fire_TPM_low_FT  = xr.open_mfdataset('../model_output/v14.3/Fire_TPM_low_FT_deposition.nc')
IKU_fire_TGM_surface = xr.open_mfdataset('../model_output/v14.3/Fire_TGM_surface_deposition.nc')
IKU_fire_TPM_surface = xr.open_mfdataset('../model_output/v14.3/Fire_TPM_surface_deposition.nc')

# -- subset time --
slice_time = slice('2015-05-01', '2016-05-01') #slice('2015-05-01','2016-04-30')
baseline = baseline.sel(time=slice_time)
IKU_fire_TGM = IKU_fire_TGM.sel(time=slice_time)
IKU_fire_TPM = IKU_fire_TPM.sel(time=slice_time)
IKU_fire_TGM_high_FT = IKU_fire_TGM_high_FT.sel(time=slice_time)
IKU_fire_TPM_high_FT = IKU_fire_TPM_high_FT.sel(time=slice_time)
IKU_fire_TGM_low_FT  = IKU_fire_TGM_low_FT.sel(time=slice_time)
IKU_fire_TPM_low_FT  = IKU_fire_TPM_low_FT.sel(time=slice_time)
IKU_fire_TGM_surface = IKU_fire_TGM_surface.sel(time=slice_time)
IKU_fire_TPM_surface = IKU_fire_TPM_surface.sel(time=slice_time)

# -- mask out ocean if bool_mask_ocean is True --
if bool_mask_ocean:
    # -- merge masks into datasets -- 
    masks = xr.open_mfdataset('../misc/masks.nc')
    baseline = xr.merge([baseline, masks])
    IKU_fire_TGM = xr.merge([IKU_fire_TGM, masks])
    IKU_fire_TPM = xr.merge([IKU_fire_TPM, masks])
    IKU_fire_TGM_high_FT = xr.merge([IKU_fire_TGM_high_FT, masks])
    IKU_fire_TPM_high_FT = xr.merge([IKU_fire_TPM_high_FT, masks])
    IKU_fire_TGM_low_FT  = xr.merge([IKU_fire_TGM_low_FT, masks])
    IKU_fire_TPM_low_FT  = xr.merge([IKU_fire_TPM_low_FT, masks])
    IKU_fire_TGM_surface = xr.merge([IKU_fire_TGM_surface, masks])
    IKU_fire_TPM_surface = xr.merge([IKU_fire_TPM_surface, masks])

    # -- mask out ocean --
    baseline = baseline.where(baseline['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TGM = IKU_fire_TGM.where(IKU_fire_TGM['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TPM = IKU_fire_TPM.where(IKU_fire_TPM['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TGM_high_FT = IKU_fire_TGM_high_FT.where(IKU_fire_TGM_high_FT['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TPM_high_FT = IKU_fire_TPM_high_FT.where(IKU_fire_TPM_high_FT['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TGM_low_FT  = IKU_fire_TGM_low_FT.where(IKU_fire_TGM_low_FT['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TPM_low_FT  = IKU_fire_TPM_low_FT.where(IKU_fire_TPM_low_FT['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TGM_surface = IKU_fire_TGM_surface.where(IKU_fire_TGM_surface['Met_FROCEAN']<0.01, drop=True)
    IKU_fire_TPM_surface = IKU_fire_TPM_surface.where(IKU_fire_TPM_surface['Met_FROCEAN']<0.01, drop=True)

if bool_exclude_ocean_Hg0_uptake:
    # -- redefine 'Total_Hg_Deposition' as the sum of 'Total_Hg_Wet_Dep' and 'Total_Hg_Dry_Dep' --
    baseline['Total_Hg_Deposition']     = baseline['Total_Hg_Wet_Dep'] + baseline['Total_Hg_Dry_Dep']
    IKU_fire_TGM['Total_Hg_Deposition'] = IKU_fire_TGM['Total_Hg_Wet_Dep'] + IKU_fire_TGM['Total_Hg_Dry_Dep']
    IKU_fire_TPM['Total_Hg_Deposition'] = IKU_fire_TPM['Total_Hg_Wet_Dep'] + IKU_fire_TPM['Total_Hg_Dry_Dep']
    IKU_fire_TGM_high_FT['Total_Hg_Deposition'] = IKU_fire_TGM_high_FT['Total_Hg_Wet_Dep'] + IKU_fire_TGM_high_FT['Total_Hg_Dry_Dep']
    IKU_fire_TPM_high_FT['Total_Hg_Deposition'] = IKU_fire_TPM_high_FT['Total_Hg_Wet_Dep'] + IKU_fire_TPM_high_FT['Total_Hg_Dry_Dep']
    IKU_fire_TGM_low_FT['Total_Hg_Deposition'] = IKU_fire_TGM_low_FT['Total_Hg_Wet_Dep'] + IKU_fire_TGM_low_FT['Total_Hg_Dry_Dep']
    IKU_fire_TPM_low_FT['Total_Hg_Deposition'] = IKU_fire_TPM_low_FT['Total_Hg_Wet_Dep'] + IKU_fire_TPM_low_FT['Total_Hg_Dry_Dep']
    IKU_fire_TGM_surface['Total_Hg_Deposition'] = IKU_fire_TGM_surface['Total_Hg_Wet_Dep'] + IKU_fire_TGM_surface['Total_Hg_Dry_Dep']
    IKU_fire_TPM_surface['Total_Hg_Deposition'] = IKU_fire_TPM_surface['Total_Hg_Wet_Dep'] + IKU_fire_TPM_surface['Total_Hg_Dry_Dep']

In [ ]:
baseline['AREA'].sum().values

In [11]:
# --------------------------------------------------------------------------------------------------------------
# make monthly deposition tables for each domain from GEOS-Chem output
# --------------------------------------------------------------------------------------------------------------

def make_monthly_table(dep_var='Total_Hg_Deposition', area_var='area [m2]'):
    ''' ''' 
    table = pd.DataFrame()
    for ds, emission_name in zip([baseline, IKU_fire_TGM, IKU_fire_TPM, IKU_fire_TGM_high_FT, IKU_fire_TPM_high_FT, IKU_fire_TGM_low_FT, IKU_fire_TPM_low_FT, IKU_fire_TGM_surface, IKU_fire_TPM_surface], 
                                ['Baseline', 'Fire (TGM)', 'Fire (TPM)', 'Fire (TGM) high FT', 'Fire (TPM) high FT', 'Fire (TGM) low FT', 'Fire (TPM) low FT', 'Fire (TGM) surface', 'Fire (TPM) surface']):
        for domain_name in domain_names:
            tmp = make_table(ds, domain_name=domain_name, emission_scenario_name=emission_name, dep_var=dep_var)
            table = pd.concat([table, tmp])
    
    # --------------------------------------------------------------------------------------------------------------
    # scale deposition based on emission uncertainty
    # --------------------------------------------------------------------------------------------------------------
    table_scaled = pd.DataFrame()
    for domain_name in domain_names:
        for emission_scenario in emission_types:
            # -- if baseline, don't replicate under all uncertainty bounds --
            if emission_scenario == 'Baseline':
                tmp = subset_table(table, match_dict={'domain name':domain_name, 'emission scenario':emission_scenario})
                tmp['emission [kg]'] = np.nan
                tmp['magnitude label'] = np.nan
                tmp['fraction TGM label'] = np.nan
                table_scaled = pd.concat([table_scaled, tmp])
            # -- if fire, replicate over uncertainty bound scenarios for speciation and magnitude --
            elif emission_scenario in ['Fire (TGM)', 'Fire (TPM)']:
                for plume_height in [' surface', ' low FT', '', ' high FT']:
                    emission_scenario_tag = emission_scenario + plume_height
                    for magnitude_tag in magnitudes:
                        for f_TGM_tag in f_TGMs:
                            tmp = scale_based_on_emission_uncertainty(table, domain_name, emission_scenario_tag, magnitude_tag, f_TGM_tag)
                            assert len(tmp) > 0, 'table is empty for domain_name={}, emission_scenario={}, magnitude_tag={}, f_TGM_tag={}'.format(domain_name, emission_scenario_tag, magnitude_tag, f_TGM_tag)
                            table_scaled = pd.concat([table_scaled, tmp])
        
    # create column 'f_FT' and name it based on value from 'emission scenario' using dictionary
    mapping = {'Fire (TGM)': 'mid', 'Fire (TPM)': 'mid',
               'Fire (TGM) low FT': 'low', 'Fire (TPM) low FT': 'low', 
               'Fire (TGM) high FT': 'high', 'Fire (TPM) high FT': 'high',
               'Fire (TGM) surface': 'surface', 'Fire (TPM) surface': 'surface',
               }

    table_scaled['fraction FT label'] = table_scaled['emission scenario'].map(mapping)

    renaming_dict = {'Baseline':'Baseline', 'Fire (TGM)':'Fire (TGM)', 'Fire (TPM)':'Fire (TPM)', 
                    'Fire (TGM) high FT': 'Fire (TGM)', 'Fire (TPM) high FT': 'Fire (TPM)',
                    'Fire (TGM) low FT' : 'Fire (TGM)', 'Fire (TPM) low FT' : 'Fire (TPM)',
                    'Fire (TGM) surface': 'Fire (TGM)', 'Fire (TPM) surface': 'Fire (TPM)'}
    # rename values in 'emission scenario' column
    table_scaled['emission scenario'] = table_scaled['emission scenario'].map(renaming_dict)

    del table

    # --------------------
    # make emission category 'Fire (all)' which is the sum of 'Fire (TGM)' and 'Fire (TPM)' for each unique combination of ['domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label']
    for domain_name in ['Alaska', 'YKD', 'IKU']:
        for magnitude_tag in ['low', 'mid', 'high']:
            for f_TGM_tag in ['low', 'mid', 'high']:
                for plume_height in ['surface', 'low', 'mid', 'high']:
                    # get total deposition for each emission scenario
                    tmp_TGM = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':'Fire (TGM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height})
                    tmp_TPM = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':'Fire (TPM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height})
                    
                    # sum the two
                    row = {'domain name':domain_name, 
                           'emission scenario':'Fire (all)', 
                           'year': tmp_TGM['year'].values, 
                           'month':tmp_TGM['month'].values, 
                           'date': tmp_TGM['date'].values,
                           'latitude min':tmp_TGM['latitude min'].values, 
                           'latitude max':tmp_TGM['latitude max'].values, 
                           'longitude min':tmp_TGM['longitude min'].values, 
                           'longitude max':tmp_TGM['longitude max'].values,
                            area_var: np.nan,
                           'total deposition [kg/mo]': tmp_TGM['total deposition [kg/mo]'] + tmp_TPM['total deposition [kg/mo]'], 
                           'total deposition [ug/m2/mo]': tmp_TGM['total deposition [ug/m2/mo]'] + tmp_TPM['total deposition [ug/m2/mo]'],
                           'emission [kg]':tmp_TGM['emission [kg]'] + tmp_TPM['emission [kg]'], 
                           'magnitude label':magnitude_tag, 
                           'fraction TGM label':f_TGM_tag, 
                           'fraction FT label':plume_height}
                    # append to table_scaled
                    table_scaled = pd.concat([table_scaled, pd.DataFrame(row)])

    # associate area with each domain
    for domain_name in domain_names:
        domain_area = baseline.sel(lat=slice(domains[domain_name]['lat_min'], domains[domain_name]['lat_max']), 
                            lon=slice(domains[domain_name]['lon_min'], domains[domain_name]['lon_max']))['AREA'].sum().values.item()
        table_scaled.loc[table_scaled['domain name']==domain_name, area_var] = domain_area

    # calculate deposition in ug/m2/yr
    table_scaled['total deposition [ug/m2/mo]'] = (table_scaled['total deposition [kg/mo]']/table_scaled[area_var])*1e9

    table_scaled = table_scaled.sort_values(['domain name', 'emission scenario', 'year', 'month', 'magnitude label', 'fraction TGM label', 'fraction FT label'])
    
    return table_scaled

# -- call function 
table_scaled = make_monthly_table(dep_var='Total_Hg_Deposition')
# save table
table_scaled.to_csv(f'../tables/deposition_table_v14_3{file_suffix}.csv', index=False)


In [ ]:
# -- for Scott, get fraction of fire-associated deposition occurring as Ocean Hg0 Uptake
print('-- ocean Hg0 uptake --')
ocean_Hg0_deposition = make_monthly_table(dep_var='FluxHg0fromAirToOcean')
for d in ['Alaska','YKD','IKU']:
    tmp = subset_table(ocean_Hg0_deposition, match_dict={'domain name':d, 'emission scenario':'Fire (all)', 'magnitude label':'mid', 'fraction TGM label':'mid', 'fraction FT label':'mid'})
    print(f'{d} total ocean Hg0 uptake (kg):', tmp['total deposition [kg/mo]'].sum())

# -- now compare to total deposition from 'Total_Hg_Dry_Dep'
print('-- dry --')
dry_dep = make_monthly_table(dep_var='Total_Hg_Dry_Dep')
for d in ['Alaska','YKD','IKU']:
    tmp = subset_table(dry_dep, match_dict={'domain name':d, 'emission scenario':'Fire (all)', 'magnitude label':'mid', 'fraction TGM label':'mid', 'fraction FT label':'mid'})
    print(f'{d} total dry dep (kg):', tmp['total deposition [kg/mo]'].sum())

# -- now compare to total deposition from 'Total_Hg_Wet_Dep'
print('-- wet --')
wet_dep = make_monthly_table(dep_var='Total_Hg_Wet_Dep')
for d in ['Alaska','YKD','IKU']:
    tmp = subset_table(wet_dep, match_dict={'domain name':d, 'emission scenario':'Fire (all)', 'magnitude label':'mid', 'fraction TGM label':'mid', 'fraction FT label':'mid'})
    print(f'{d} total wet dep (kg):', tmp['total deposition [kg/mo]'].sum())

# -- now compare to total deposition from 'Total_Hg_Deposition' 
# -- this has been redefined to be the sum of 'Total_Hg_Wet_Dep' and 'Total_Hg_Dry_Dep'
print('-- wet + dry --')
total_dep = make_monthly_table(dep_var='Total_Hg_Deposition')
for d in ['Alaska','YKD','IKU']:
    tmp = subset_table(total_dep, match_dict={'domain name':d, 'emission scenario':'Fire (all)', 'magnitude label':'mid', 'fraction TGM label':'mid', 'fraction FT label':'mid'})
    print(f'{d} total dep (wet+dry) (kg):', tmp['total deposition [kg/mo]'].sum())

In [ ]:
subset_table(table_scaled, match_dict={'domain name':'Alaska', 'emission scenario':'Fire (TGM)', 'magnitude label':'low', 'fraction TGM label':'low', 'fraction FT label':'mid'})

In [ ]:
def make_deposition_bar_chart(table_scaled, domain_name, area_var='area [m2]'):
    """ Make a bar chart of total deposition for each month of the year. """
    col_name = 'total deposition [kg/mo]'
    time = table_scaled['date'].unique()
    time = pd.to_datetime(time)

    y_baseline = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':'Baseline'})[col_name].values

    y_fire_all = []
    y_fire_TGM_all = []
    y_fire_TPM_all = []

    for plume_height in ['surface', 'low', 'mid', 'high']:
        for magnitude_tag in ['low', 'mid', 'high']:
            for f_TGM_tag in ['low', 'mid', 'high']:
                y_fire_TGM = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':'Fire (TGM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label': plume_height})[col_name]
                y_fire_TPM = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':'Fire (TPM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label': plume_height})[col_name]
                
                y_fire_TGM_all.append(y_fire_TGM.values)
                y_fire_TPM_all.append(y_fire_TPM.values)
                y_fire_all.append( (y_fire_TGM+y_fire_TPM).values )

                if magnitude_tag == 'mid' and f_TGM_tag == 'mid':
                    y_fire_TGM_mid = y_fire_TGM.values
                    y_fire_TPM_mid = y_fire_TPM.values
                    y_fire_mid = (y_fire_TGM+y_fire_TPM).values

    # -- get max and min of all uncertainty conditions
    # - TGM -
    y_fire_TGM_high = np.max(y_fire_TGM_all, axis=0)
    y_fire_TGM_low = np.min(y_fire_TGM_all, axis=0)
    # - TPM -
    y_fire_TPM_high = np.max(y_fire_TPM_all, axis=0)
    y_fire_TPM_low = np.min(y_fire_TPM_all, axis=0)
    # - TGM + TPM -
    y_fire_high = np.max(y_fire_all, axis=0)
    y_fire_low = np.min(y_fire_all, axis=0)

    x = np.arange(len(time))
    offset = 0.2
    width = 0.2
    fig, ax = plt.subplots(figsize=(6,3))

    # - plot baseline -
    ax.bar(x, y_baseline, width=width, facecolor=color_dict['brown'], edgecolor='k', label='Background')
    # - plot TGM + TPM -
    ax.bar(x+offset, y_fire_mid, width=width, facecolor=color_dict['red'], edgecolor='k', label='Fire (TGM + TPM)')
    ax.errorbar(x+offset, y_fire_mid, yerr=[y_fire_mid-y_fire_low, y_fire_high-y_fire_mid], 
                fmt='none', ecolor='0.2', lw=1, capsize=3) # add error bars
    offset += width
    # - plot TGM -
    ax.bar(x+offset, y_fire_TGM_mid, width=width, facecolor=color_dict['blue'], edgecolor='k', label='Fire (TGM)')
    ax.errorbar(x+offset, y_fire_TGM_mid, yerr=[y_fire_TGM_mid-y_fire_TGM_low, y_fire_TGM_high-y_fire_TGM_mid], 
                fmt='none', ecolor='0.2', lw=1, capsize=3) # add error bars
    offset += width
    # - plot TPM -
    ax.bar(x+offset, y_fire_TPM_mid, width=width, facecolor=color_dict['green'], edgecolor='k', label='Fire (TPM)')
    ax.errorbar(x+offset, y_fire_TPM_mid, yerr=[y_fire_TPM_mid-y_fire_TPM_low, y_fire_TPM_high-y_fire_TPM_mid], 
               fmt='none', ecolor='0.2', lw=1, capsize=3) # add error bars

    ax.set_xticks(x+(offset/2))
    # format xtick labels as month names
    ax.set_xticklabels(time.strftime('%b'))
    ax.set_xlim(-0.5, 4.5)
    ax.set_ylabel('Hg Deposition (kg mo$\mathrm{^{-1}}$)')
    ax.set_xticks(ticks=[], minor=True)

    # add legend
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, 
              labels, loc='upper left', 
              edgecolor='None',
              ncol=1,
              fontsize=8,
              labelcolor='0.3',
              )

    # label other side with deposition in ug/m2/mo
    ax2 = ax.twinx()
    ymin, ymax = ax.get_ylim()
    # convert kg/mo to ug/m2/mo
    area = subset_table(table_scaled, {'domain name':domain_name})[area_var].unique().item()
    ax2.set_ylim((ymin/area)*1e9, (ymax/area)*1e9)
    ax2.set_ylabel('Hg Deposition ($\mathrm{\mu}$g m$^\mathrm{-2}$ mo$\mathrm{^{-1}}$)')

    coords = domains[domain_name]
    if domain_name == 'IKU':
        ax.text(0.98, 0.97, 
                s=f'Domain: 61.25 - 61.5 $\degree$N; 162.5 - 163.125 $\degree$W', 
                verticalalignment='top', horizontalalignment='right', transform=ax.transAxes,
                fontsize=8, color='0.3')

        ax.set_ylim(0, 18)
        ax.set_yticks(np.arange(0, 18.1, 3))
        ax.set_yticks(np.arange(0,18.1, 1), minor=True)

        ax2.set_yticks(np.arange(0, 6.1, 0.5), minor=True)

    # turn off grid
    ax.grid(False)
    ax2.grid(False)
    
    plt.savefig(f'../figures/{domain_name}_domain/total_deposition_bar_chart_14_3{file_suffix}.pdf', bbox_inches='tight')
    plt.savefig(f'../figures/{domain_name}_domain/total_deposition_bar_chart_14_3{file_suffix}.png', bbox_inches='tight', dpi=1000)

    return fig, ax

# --------------------------------------------------------------------------------------------------------------
# make bar charts for IKU and YKD domains. effect is too small to see in Alaska domain.
# --------------------------------------------------------------------------------------------------------------
fig, ax = make_deposition_bar_chart(table_scaled, domain_name='IKU')
fig, ax = make_deposition_bar_chart(table_scaled, domain_name='YKD')

In [17]:
deposition_totals = pd.DataFrame()
# make summary deposition table
for domain_name in domain_names:
    for emission_name in emission_types:

        if emission_name == 'Baseline':
            tmp = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':emission_name})
            row = {'domain name':domain_name, 'emission scenario':emission_name, 'total deposition [kg]':tmp['total deposition [kg/mo]'].sum().item(), 'emission [kg]':np.nan, 'magnitude label':np.nan, 'fraction TGM label':np.nan}
            deposition_totals = pd.concat([deposition_totals, pd.DataFrame(row, index=[0])])
        
        elif emission_name in ['Fire (TGM)', 'Fire (TPM)']:
            for plume_height in plume_heights:
                #emission_scenario_tag = emission_name + plume_height
                for magnitude_tag in magnitudes:
                    for f_TGM_tag in f_TGMs:
                        #print('domain:', domain_name, 'emission type:', emission_name, 'emission magnitude:', magnitude_tag, 'f_TGM:', f_TGM_tag, 'plume height:', plume_height)
                        tmp = subset_table(table_scaled, {'domain name':domain_name, 'emission scenario':emission_name, 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label': plume_height})
                        row = {'domain name':domain_name, 'emission scenario':emission_name, 'total deposition [kg]':tmp['total deposition [kg/mo]'].sum().item(), 'emission [kg]':tmp['emission [kg]'].unique().item(), 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height}
                        #print(domain_name, emission_name, magnitude_tag, f_TGM_tag, np.round(tmp['total deposition [kg/mo]'].sum().item(),1))
                        deposition_totals = pd.concat([deposition_totals, pd.DataFrame(row, index=[0])])

# make emission category 'Fire (all)' which is the sum of 'Fire (TGM)' and 'Fire (TPM)' for each unique combination of ['domain name', 'magnitude label', 'fraction TGM label', 'fraction FT label']
for domain_name in domain_names:
    for magnitude_tag in magnitudes:
        for f_TGM_tag in f_TGMs:
            for plume_height in plume_heights:
                # get total deposition for each emission scenario
                tmp_TGM = subset_table(deposition_totals, {'domain name':domain_name, 'emission scenario':'Fire (TGM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height})
                tmp_TPM = subset_table(deposition_totals, {'domain name':domain_name, 'emission scenario':'Fire (TPM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height})
                # sum the two
                row = {'domain name':domain_name, 'emission scenario':'Fire (all)', 'total deposition [kg]':tmp_TGM['total deposition [kg]'].item() + tmp_TPM['total deposition [kg]'].item(), 'emission [kg]':tmp_TGM['emission [kg]'].item() + tmp_TPM['emission [kg]'].item(), 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height}
                # append to deposition_totals
                deposition_totals = pd.concat([deposition_totals, pd.DataFrame(row, index=[0])])

# associate area with each domain
for domain_name in domain_names:
    domain_area = baseline.sel(lat=slice(domains[domain_name]['lat_min'], domains[domain_name]['lat_max']), 
                        lon=slice(domains[domain_name]['lon_min'], domains[domain_name]['lon_max']))['AREA'].sum().values.item()
    deposition_totals.loc[deposition_totals['domain name']==domain_name, area_var] = domain_area

# calculate deposition in ug/m2/yr
deposition_totals['total deposition [ug/m2/yr]'] = (deposition_totals['total deposition [kg]']/deposition_totals[area_var])*1e9

deposition_totals.to_csv(f'../tables/deposition_annual_v14_3{file_suffix}.csv', index=False)

In [ ]:
sel = deposition_totals[(deposition_totals['domain name']=='Alaska') & (deposition_totals['emission scenario']=='Fire (all)')]
sel = sel[sel['fraction FT label']!='surface']
sel['total deposition [kg]'].max()

In [ ]:
var_template = {'magnitude label': 'mid', 'fraction TGM label':'mid', 'fraction FT label':'mid'}

output_table = pd.DataFrame()
for v in ['magnitude label', 'fraction TGM label', 'fraction FT label']:
    tag_list = ['low', 'mid', 'high']
    if v == 'fraction FT label':
        tag_list = ['surface', 'low', 'mid', 'high']
    for tag in tag_list:
        var_dict = var_template.copy()
        var_dict[v] = tag
        row = {'variable': [v], 'parameter value': [tag]}
        for domain in ['Alaska', 'YKD', 'IKU']:
            var_dict['domain name'] = domain
            var_dict['emission scenario'] = 'Fire (all)'

            D = subset_table(deposition_totals, var_dict)
            row[domain] = [D['total deposition [kg]'].item()]

        output_table = pd.concat([output_table, pd.DataFrame(row)])

output_table.to_csv(f'../tables/Table_S7{file_suffix}.csv', index=False)

output_table

In [ ]:
for variable in ['magnitude label', 'fraction TGM label', 'fraction FT label']:

    sel = output_table[output_table['variable']==variable]

    # get relative change in deposition under low vs. mid scenario
    low = sel[sel['parameter value']=='low']
    mid = sel[sel['parameter value']=='mid']
    high = sel[sel['parameter value']=='high']

    # drop parameter value column
    low = low.drop(columns='parameter value')
    mid = mid.drop(columns='parameter value')
    high = high.drop(columns='parameter value')

    if variable == 'fraction FT label':
        surface = sel[sel['parameter value']=='surface']
        surface = surface.drop(columns='parameter value')
        delta_surface = (surface.set_index('variable') - mid.set_index('variable'))/mid.set_index('variable')
        delta_surface_pct = delta_surface*100

    delta_low = (low.set_index('variable') - mid.set_index('variable'))/mid.set_index('variable')
    delta_high = (high.set_index('variable') - mid.set_index('variable'))/mid.set_index('variable')
    delta_low_pct = delta_low*100
    delta_high_pct = delta_high*100

    print(variable)
    print('low')
    display(delta_low)
    print('high')
    display(delta_high)
    if variable == 'fraction FT label':
        print('surface')
        display(delta_surface)
    print('------')


In [21]:
# ['magnitude_label', 'f_TGM', 'plume_height']
plume_height = 'mid'

# [('magnitude label', 'fraction TGM label')]
tag_pairs = [('mid', 'low'),
             ('low','low'),
             ('low', 'high'),
             ('high','low'),
             ('high','high')]

summary_table = pd.DataFrame()
for domain in ['Alaska', 'YKD', 'IKU']:
    area = table_scaled[table_scaled['domain name']==domain][area_var].unique().item()

    baseline = subset_table(deposition_totals, {'domain name':domain, 'emission scenario':'Baseline'})

    for magnitude_tag, f_TGM_tag in tag_pairs:
        TGM = subset_table(deposition_totals, {'domain name':domain, 'emission scenario':'Fire (TGM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height})
        TPM = subset_table(deposition_totals, {'domain name':domain, 'emission scenario':'Fire (TPM)', 'magnitude label':magnitude_tag, 'fraction TGM label':f_TGM_tag, 'fraction FT label':plume_height})

        row = {'domain name':domain, 
                area_var: area*1e-6, 
                'magnitude label':magnitude_tag, 
                'fraction TGM label':f_TGM_tag, 
                'fraction FT label':plume_height,
                'E TGM [kg]':TGM['emission [kg]'].unique().item(), 
                'E TPM [kg]':TPM['emission [kg]'].unique().item(),
                'baseline [kg]':baseline['total deposition [kg]'].unique().item(), 
                'D TGM [kg]':TGM['total deposition [kg]'].unique().item(), 
                'D TPM [kg]':TPM['total deposition [kg]'].unique().item(),
                }

        summary_table = pd.concat([summary_table, pd.DataFrame(row, index=[0])])    

round_cols = ['E TGM [kg]', 'E TPM [kg]', 'baseline [kg]', 'D TGM [kg]', 'D TPM [kg]']
summary_table[round_cols] = summary_table[round_cols].round(1)

summary_table = pd.merge(summary_table, table_scaled[['domain name', 'latitude min', 'latitude max', 'longitude min', 'longitude max']].drop_duplicates(), on='domain name', how='left')
summary_table['D Total [kg]'] = summary_table['D TGM [kg]'] + summary_table['D TPM [kg]']
summary_table['E Total [kg]'] = summary_table['E TGM [kg]'] + summary_table['E TPM [kg]']
summary_table['Scenario (E/f_TGM)'] = summary_table['magnitude label'] + '/' + summary_table['fraction TGM label']
summary_table['f_TGM'] = summary_table['fraction TGM label'].map({'mid':0.935, 'low':0.935, 'high': 0.992})
col_order = ['domain name', area_var, 'baseline [kg]', 'Scenario (E/f_TGM)', 'f_TGM', 'E Total [kg]', 
             'E TGM [kg]', 'E TPM [kg]', 'D Total [kg]', 'D TGM [kg]', 'D TPM [kg]', 
             ]
summary_table = summary_table[col_order]
# replace '[m2]' with '[km2]' in area column
summary_table = summary_table.rename(columns={area_var:area_var.replace('m2','km2')})

summary_table.to_csv(f'../tables/Table_1{file_suffix}.csv', index=False)

In [ ]:
summary_table

In [ ]:
summary_table['baseline [ug/m2/yr]'] = (summary_table['baseline [kg]']*1e9)/(summary_table['area [km2]']*1e3*1e3)
summary_table

In [ ]:
summary_table[summary_table['Scenario (E/f_TGM)']=='mid/low']

In [ ]:
deposition_totals